## **Criando meu primeito Catálogo**

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS demo_catalog
COMMENT 'Catálogo de demonstração criado'

## Criando um SCHEMA (database) no catalogo 

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo_catalog.bronze
COMMENT 'Schema de Bronze para dados brutos'

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo_catalog.silver
COMMENT 'Schema de silver para dados tratados'

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS demo_catalog.gold
COMMENT 'Schema de Gold para dados que serão usados em negócios'

## **Criando uma TABELA**

In [0]:
%sql
CREATE TABLE demo_catalog.bronze.sales(
  sale_id INT,
  product STRING,
  quantity INT,
  price DOUBLE,
  sale_date DATE
)
USING DELTA
COMMENT'Tabela de vendas brutas no formato Delta';

In [0]:
%sql
select * from demo_catalog.bronze.sales

In [0]:
%sql
INSERT INTO demo_catalog.bronze.sales (sale_id, product, quantity, price, sale_date)
VALUES
-- Grupo 1: Notebook
(1,  'Notebook Dell', 1,  5500.00, DATE '2025-10-01'),
(2,  'Notebook Dell', 2,  5200.00, DATE '2025-10-02'),
(3,  'Notebook Dell', 1,  5300.00, DATE '2025-10-03'),
(4,  'Notebook Dell', 1,  5400.00, DATE '2025-10-04'),
(5,  'Notebook Dell', 1,  5600.00, DATE '2025-10-05'),

-- Grupo 2: Monitor
(6,  'Monitor LG 27"', 2, 1200.00, DATE '2025-10-06'),
(7,  'Monitor LG 27"', 1, 1150.00, DATE '2025-10-07'),
(8,  'Monitor LG 27"', 3, 1180.00, DATE '2025-10-08'),
(9,  'Monitor LG 27"', 2, 1220.00, DATE '2025-10-09'),
(10, 'Monitor LG 27"', 1, 1250.00, DATE '2025-10-10'),

-- Grupo 3: Headset
(11, 'Headset HyperX',  2,  600.00, DATE '2025-10-11'),
(12, 'Headset HyperX',  1,  590.00, DATE '2025-10-12'),
(13, 'Headset HyperX',  3,  620.00, DATE '2025-10-13'),
(14, 'Headset HyperX',  2,  610.00, DATE '2025-10-14'),
(15, 'Headset HyperX',  1,  630.00, DATE '2025-10-15');


## Criando uma View com calculos 

In [0]:
%sql
CREATE OR REPLACE VIEW demo_catalog.bronze.vw_sales_summary AS
SELECT
  product,
  SUM(quantity) AS total_sold,
  ROUND(SUM(quantity * price), 2) AS revenue
  FROM demo_catalog.bronze.sales
  GROUP BY product

## Criando um VOLUME(arquivos)

In [0]:
%sql
CREATE VOLUME demo_catalog.bronze.raw_files
COMMENT 'Volume para arquivos brutos de ingestão inicial'

## Ler e exibir os dados diretamente de .cvs, json e parquet

In [0]:
 # -- Parquet --
 df_parquet = spark.read.parquet("/Volumes/demo_catalog/bronze/raw_files/dados.parquet")
 print('PARQUET:')
 df_parquet.show(truncate=False)

 # -- CSV ---
 df_csv = (
     spark.read
     .option("header", True)
     .option("inferSchema", True)
     .csv("/Volumes/demo_catalog/bronze/raw_files/dados.csv")
 )
 print('CSV:')
 df_csv.show(truncate=False)

 # == JSON ==
 df_json = spark.read.json("/Volumes/demo_catalog/bronze/raw_files/dados.json")
 print('JSON:')
 df_json.show(truncate=False)

## Salvando em views

In [0]:
# Ler parquet 
df_parquet = spark.read.parquet("/Volumes/demo_catalog/bronze/raw_files/dados.parquet")
df_parquet.createOrReplaceTempView("vw_parquet")

# Ler CSV
df_csv = (
     spark.read
     .option("header", True)
     .option("inferSchema", True)
     .csv("/Volumes/demo_catalog/bronze/raw_files/dados.csv")
 )
df_csv.createOrReplaceTempView("vw_csv")

# Ler JSON
df_json = spark.read.json("/Volumes/demo_catalog/bronze/raw_files/dados.json")
df_json.createOrReplaceTempView("vw_json")

## Criando uma função SQL no Unity Catalog

In [0]:
%sql
CREATE OR REPLACE FUNCTION demo_catalog.bronze.calc_bonus(salario DOUBLE, percentual DOUBLE)
RETURNS DOUBLE
COMMENT 'Calcula o bônus de um funcionário'
RETURN salario * (percentual / 100.0);

In [0]:
%sql
USE CATALOG demo_catalog;
USE SCHEMA bronze;

In [0]:
%sql
SELECT
    nome,
    salario,
    calc_bonus(salario, 10) AS bonus_id
FROM vw_parquet;

## Criando um model via mlflow

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS demo_catalog;
CREATE SCHEMA IF NOT EXISTS demo_catalog.ml
COMMENT 'Schema para registrar modelos de ML (Unity Catalog)';

In [0]:
# %pip install mlflow scikit-learn --quiet  # se precisar

import pandas as pd
import mlflow, mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature

# === Dados como DataFrame (facilita a signature com nomes de colunas) ===
iris = load_iris(as_frame=True)
X, y = iris.data, iris.target  # X é DataFrame, y é Series
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# === Modelo ===
clf = RandomForestClassifier(n_estimators=50, random_state=42).fit(X_train, y_train)

# === Signature + input_example (obrigatórios para UC Models) ===
input_example = X_train.iloc[:3]
signature = infer_signature(X_train, clf.predict(X_train))

# === Registrar no Unity Catalog ===
mlflow.set_registry_uri("databricks-uc")

with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="model",
        registered_model_name="demo_catalog.ml.iris_model",  # ..
        signature=signature,
        input_example=input_example
    )

print("✅ Modelo registrado com signature: demo_catalog.ml.iris_model")